In [116]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [117]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)
print("-")

-


In [118]:
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage

def imshow(img):
    img = img /2 + 0.5
    np_img = img.numpy()
    plt.imshow(np.transpose(np_img, (1,2,0)))
    # plt.show()


In [134]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, self.num_flat_features(x)) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x): 
        size = x.size()[1:] 
        num_features = 1 
        for s in size: 
            num_features *= s 
        return num_features


In [121]:
train_loader = DataLoader(train_data, batch_size = 16, shuffle = True, num_workers = 4)
test_loader = DataLoader(test_data, batch_size = len(test_data))

In [136]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')
print(torch.cuda.is_available())

cpu is available
False


In [137]:
cnn.train()  # 학습을 위함
for epoch in range(5):
  for index, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()  # 기울기 초기화
    output = cnn(data)
    print(output)
    loss = criterion(output, target)
    loss.backward()  # 역전파
    optimizer.step()

    if index % 100 == 0:
      print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

    [ -2.7930,  14.2550,   1.0842,  -5.1604,   3.3148,  -6.0894,  -4.7838,
           6.5041,  -0.6489,   0.3942],
        [ -3.6448,  -6.6032,   6.6974,   3.6205,  -5.3581,  -0.0834,  -3.1347,
          -3.1153,  21.3664,   0.4437],
        [ -7.9593,   0.0310,   1.9123,  16.9431,  -5.7750,  -0.9671, -11.4226,
           3.5660,   3.4909,   1.6401]], grad_fn=<AddmmBackward>)
tensor([[  3.5040,  -7.6490,  -0.4290,  -1.6884,  -4.4954,   5.5515,  15.7414,
          -8.1349,   7.7689,  -4.1289],
        [ -5.4662,   1.1298,   4.3331,   6.3005,  -0.4109,  -7.3403, -14.3148,
          17.3340,  -0.6086,   0.4817],
        [ -5.8702,   6.4029,   3.1862,   6.2097,  -4.7276,  -5.1897, -12.1311,
          14.8862,  -0.4397,   1.4281],
        [ -4.2775,   2.4397,  14.6742,   3.7410,  -5.3392,  -4.5457,  -8.0265,
           6.9335,   2.4526,  -6.8850],
        [  3.2970,  -4.2004,  -0.5767,  -3.3217,  -1.4050,   5.5319,  14.4109,
          -4.6723,   0.2963,  -4.2320],
        [ -5.0823,   1.689

In [138]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    output = cnn(data)
    test_loss += criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0396, Accuracy: 9871/10000 (99%)

